In [126]:
import json
import pandas as pd
import numpy as np
import qgrid
from _vars import *
import os
import sys
import zipfile

In [156]:
zip_filepath="json_archive.zip"  #input parameter
target_dir="extracted_files/"  #input parameter
files_to_extract = ['40171448_final.json', '10171448_final.json', '14171448_final.json'] #WRITE HERE FILENAMES

In [157]:
def unzip(zip_filepath, target_dir, files_to_extract):
    """
    Unzips the files from a zip file and puts them in the given destination (can be a non-existing folder). 
    Out put can be filtered by providing the file extension.
    :param zip_file: zip file to extract.
    :param target_dir: destination folder.
    :param files_to_extract: list of filenames to extract
    :return: nothing
    """
    if not os.path.exists(target_dir):
        os.makedirs(target_dir)

    try:
        zfile = zipfile.ZipFile(zip_filepath, 'r')
        for filename in files_to_extract:
            print('Extracting {f} to {d}'.format(f=filename, d=target_dir))
            zfile.extract(filename, target_dir)

    except zipfile.BadZipfile:
        print("Cannot extract {f}: Not a valid zipfile (BadZipfile Exception)".format(f=zip_filepath))
        
unzip(zip_filepath, target_dir, files_to_extract)

Extracting 40171448_final.json to extracted_files/
Extracting 10171448_final.json to extracted_files/
Extracting 14171448_final.json to extracted_files/


In [143]:
'''Reading a file and putting the data into dataframe'''
with open('1206_final.json') as f:
    data = json.load(f)
    df = pd.DataFrame(data)

In [152]:
'''Extracting the identificators of references and their types'''
def extract_id(token_string):
    for _id in dict_of_ids.keys():
        token_id = re.search(dict_of_ids[_id] , token_string)
        if token_id is not None:
            token_id = token_id.group()
            ref = re.compile("([a-zA-Z=:|]*)([0-9].+)").match(token_id).groups() 
            return (_id, ref[1])
        
        
df['ref_ids'] = np.empty((len(df), 0)).tolist()
df['ref_id_ins'] = None
df['ref_ids_type'] = np.empty((len(df), 0)).tolist()
for i in range(len(df)):
    ref_list = []
    for seq in df['change_sequence'][i]:
        ref = extract_id("".join(seq['tokens']))
        if ref != None and df.loc[i, 'ref_id_ins'] == None:
            df.loc[i, 'ref_id_ins'] = seq['time']
        if ref != None:
            ref_list.append((ref[0].replace("=", ""), ref[1]))
    ref_list = set(ref_list)
    df.at[i, 'ref_ids'] = [x[1] for x in ref_list]
    df.at[i, 'ref_ids_type'] = [x[0] for x in ref_list]
    

In this example we are extracting the dates of reference insertion, date of id insertion and final reference deletion for each reference by its id:

In [153]:
def getting_data(df):
    df_upt = pd.DataFrame(df[['ref_ids','ref_ids_type', 'ref_id_ins']])
    df_upt['ins_time'] = df['first_rev_time']
    df_upt['del_time'] = 'None'
    for i in df_upt.index:
        if df['deleted'][i]:
            df_upt['del_time'][i] = df['del_time'][i][-1]
    return df_upt
        
df_upt = getting_data(df)
qgrid.show_grid(getting_data(df))

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…